## Simple Gen AI App using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

## Data Ingestion 

From the website we need to scrape the data.

In [3]:
from langchain_community.document_loaders import WebBaseLoader

c:\Users\Karan\Desktop\agentic-ai-langchain-langgraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader.load()

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content="Manage billing in your account - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Releases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up bi

In [5]:
loader

In [6]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content="Manage billing in your account - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Releases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up bi

In [12]:
## Load Data > Docs > Divide into Chunks > Create Vectors > Vector Embeddings>Store it into VectorStore DB

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)



In [13]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content="Manage billing in your account - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Releases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up bi

In [14]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [16]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [17]:
vectorstoredb

In [22]:
## Query from a Vector DB
query = "How to manage data retention?"

In [23]:
result = vectorstoredb.similarity_search(query)

In [24]:
result[0].page_content

'The extended data retention limit can cause features other than tracing to stop working once reached. If you plan to use this feature, read more about its functionality and side effects.\n\u200bOther methods of managing traces\n\u200bChange project-level default retention\nData retention settings are adjustable per tracing project.\nNavigate to Projects > Your project name > Select Retention and select the desired default retention. This will only affect retention (and pricing) for traces going forward.'

In [28]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [31]:
## Retrieval Chain, Document Chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
    """
    )

doc_chain = create_stuff_documents_chain(llm,prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n    '), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.co

In [33]:
from langchain_core.documents import Document
doc_chain.invoke({
    "input": "LangSmith has 2 usage limits. What are they?",
    "context": [Document(page_content="LangSmith has 2 usage limits: total traces and extended traces")],
})

'LangSmith imposes two types of usage limits: one for total traces and another for extended traces.'

---------------------------------------------------------------------------
We want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question